In [ ]:
# !PROJECT_ID="" 
# !REGION=us-central1  
# !REPOSITORY=black-friday-v2 
# !IMAGE=black-friday-image 


%env PROJECT_ID= 
%env REGION=us-central1  
%env REPOSITORY=black-friday-v1 
%env IMAGE=black-friday-image 

In [ ]:
! pwd

In [ ]:
! gcloud beta artifacts repositories create $REPOSITORY  \
 --repository-format=docker \
 --location=$REGION

In [ ]:
! docker push $REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY/$IMAGE


In [ ]:
! gcloud beta ai models upload \
  --region=us-central1 \
  --display-name=black-friday-model \
  --container-image-uri=$REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY/$IMAGE \
  --container-ports=5005 \
  --container-health-route=/healthz \
  --container-predict-route=/predict

In [ ]:
!gcloud beta ai models list \
  --region=us-central1 \
  --filter=display_name=black-friday-model

In [ ]:

!gcloud beta ai endpoints create \
  --region=us-central1 \
  --display-name=black-friday-test-v2

In [ ]:
# errors out over here 
!gcloud beta ai endpoints deploy-model 7245157104439263232 \
  --region=us-central1 \
  --model=1949385737535225856 \
  --display-name=black-friday-model1 \
  --machine-type=n1-standard-4 \
  --min-replica-count=1 \
  --max-replica-count=2 \
  --traffic-split=0=100

In [ ]:
ENDPOINT_ID="4047601369006211072"
INPUT_DATA_FILE="INPUT-JSON"

In [ ]:
!curl -X POST -d "input.json" -H "Content-Type: application/json" http://localhost:5005/predict


In [ ]:
import requests
import json


In [ ]:
url = 'http://0.0.0.0:5005/predict'
d = json.load(open("input.json", "rb"))
# data = json.load()
payload = {'instances': d}
# y_predict = requests.post(url, json=payload)
r = requests.post(url,json=payload)


In [ ]:
r.status_code

In [ ]:
r.text

In [ ]:
r.json

In [ ]:
import pickle
model = pickle.load(open("model.pkl", "rb"))


In [ ]:
model.predict([data]).tolist()[0]

In [ ]:
! curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/mwpmltr/locations/us-central1/endpoints/4047601369006211072:predict \
-d "@$input.json"